In [52]:
import numpy as np
import pandas as pd
import altair as alt

In [53]:
# load in data
opioid_deaths = pd.read_parquet("/Users/erikafox/Desktop/pds2021-opioids-pds6/10_modified_data/vital_pop.parquet", engine='fastparquet')
opioid_buyers = pd.read_parquet("/Users/erikafox/Desktop/pds2021-opioids-pds6/10_modified_data/opi_pop.parquet", engine='fastparquet')

In [54]:

opioid_buyers=opioid_buyers.rename(columns={"MME per cap":"MME_per_cap"})
opioid_deaths=opioid_deaths.rename(columns={"vital deaths per cap":"deaths_per_cap"})
opioid_buyers.sample(10)


,BUYER_STATE,BUYER_COUNTY,fips,year,MME,Population,MME_per_cap,treatment
index,,,,,,,,
4261,MT,BROADWATER,30007,2007,4.951330e+05,5089.0,97.294748,0
7671,TX,MILLS,48333,2008,4.966663e+05,4876.0,101.859377,0
306,AL,HOUSTON,01069,2006,2.987526e+07,95846.0,311.700621,0
4492,MT,MUSSELSHELL,30065,2011,2.331126e+06,4747.0,491.073457,0
4388,MT,HILL,30041,2006,4.613690e+06,15981.0,288.698477,0
4117,MS,TIPPAH,28139,2007,6.103940e+06,21884.0,278.922523,0
5299,OK,LOGAN,40083,2007,7.811893e+06,38709.0,201.810756,0
692,FL,CLAY,12019,2014,9.541884e+07,198570.0,480.529976,1
2727,ID,FRANKLIN,16041,2009,2.441879e+06,12757.0,191.414792,0


In [55]:
opioid_deaths.sample(10)

,STNAME,CTYNAME,fips,Year,Deaths,deaths_per_cap,treatment
index,,,,,,,
6420,Kansas,Sedgwick County,20173,2011,45,0.000090,0
5667,Oklahoma,Pottawatomie County,40125,2012,11,0.000156,0
3886,Kentucky,Laurel County,21125,2015,13,0.000217,0
6591,California,Sonoma County,06097,2013,31,0.000063,0
1133,Maryland,Cecil County,24015,2009,17,0.000169,0
7120,Oklahoma,Tulsa County,40143,2012,108,0.000176,0
2630,West Virginia,Greenbrier County,54025,2015,10,0.000281,0
717,Florida,Broward County,12011,2007,282,0.000164,1
565,Idaho,Bonneville County,16019,2011,12,0.000113,0


In [56]:
## Nick's code for confidence bands 

def get_reg_fit(data, yvar, xvar, alpha=0.05,col="blue"):
    import statsmodels.formula.api as smf

    # Grid for predicted values
    x = data.loc[pd.notnull(data[yvar]), xvar]
    xmin = x.min()
    xmax = x.max()
    step = (xmax - xmin) / 100
    grid = np.arange(xmin, xmax + step, step)
    predictions = pd.DataFrame({xvar: grid})

    # Fit model, get predictions
    model = smf.ols(f"{yvar} ~ {xvar}", data=data).fit()
    model_predict = model.get_prediction(predictions[xvar])
    predictions[yvar] = model_predict.summary_frame()["mean"]
    predictions[["ci_low", "ci_high"]] = model_predict.conf_int(alpha=alpha)

    # Build chart
    reg = alt.Chart(predictions).mark_line(color=col).encode(x=xvar, y=yvar)
    ci = (
        alt.Chart(predictions)
        .mark_errorband(color=col)
        .encode(
            x=xvar,
            y=alt.Y("ci_low", title=yvar),
            y2="ci_high",
        )
    )
    chart = ci + reg
    return predictions, chart

# Florida plots

In [57]:
# Creating florida buyers df

opioid_buyers_fl=opioid_buyers[opioid_buyers["BUYER_STATE"]=="FL"]


opioid_buyers_fl["policy_years"]= opioid_buyers_fl["year"]-2010


fl_opi_before=opioid_buyers_fl[opioid_buyers_fl["policy_years"]<0]
fl_opi_after=opioid_buyers_fl[opioid_buyers_fl["policy_years"]>=0]

/var/folders/5b/sjkc4b8n0hs41zd03hxr66100000gn/T/ipykernel_47842/930905532.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  opioid_buyers_fl["policy_years"]= opioid_buyers_fl["year"]-2010


In [58]:
#create vertical line for plots
data = pd.DataFrame({"a": [0]})
sep_line = (alt.Chart(data).mark_rule(color="black", strokeDash=[10, 10]).encode(x="a"))

In [59]:
# florida buyers pre/post plot
before=alt.Chart(fl_opi_before,title="Pre-Post Policy: Florida, Opioids per cap").encode(x="policy_years", y="MME_per_cap")
base=before.transform_regression("policy_years","MME_per_cap").mark_line() 

fit, before_line = get_reg_fit(
    fl_opi_before, yvar="MME_per_cap", xvar="policy_years", alpha=0.05
)
fit, after_line = get_reg_fit(
    fl_opi_after, yvar="MME_per_cap", xvar="policy_years", alpha=0.05
)

fl_chart_opioid=base+before_line+after_line+sep_line
fl_chart_opioid


alt.LayerChart(...)

In [60]:
# Creating florida deathd df

opioid_deaths_fl=opioid_deaths[opioid_deaths["STNAME"]=="Florida"]

opioid_deaths_fl["policy_years"]= opioid_deaths_fl["Year"]-2010


fl_death_before=opioid_deaths_fl[opioid_deaths_fl["policy_years"]<0]
fl_death_after=opioid_deaths_fl[opioid_deaths_fl["policy_years"]>=0]

/var/folders/5b/sjkc4b8n0hs41zd03hxr66100000gn/T/ipykernel_47842/3666732951.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  opioid_deaths_fl["policy_years"]= opioid_deaths_fl["Year"]-2010


In [61]:
# florida deaths pre/post plot
before=alt.Chart(fl_death_before,title="Pre-Post Policy: Florida, Overdose Deaths per cap").encode(x="policy_years", y="deaths_per_cap")
base=before.transform_regression("policy_years","deaths_per_cap").mark_line() 

fit, before_line = get_reg_fit(
    fl_death_before, yvar="deaths_per_cap", xvar="policy_years", alpha=0.05
)
fit, after_line = get_reg_fit(
    fl_death_after, yvar="deaths_per_cap", xvar="policy_years", alpha=0.05
)

fl_chart_death=base+before_line+after_line+sep_line
fl_chart_death

alt.LayerChart(...)

In [62]:
# Creating control dataframes
states = ["SC","AL","GA","MS"]
states2 = ["South Carolina","Alabama","Georgia","Mississippi"]

opioid_buyers_con=opioid_buyers[opioid_buyers["BUYER_STATE"].isin(states)]
opioid_buyers_con.sample(10)
opioid_buyers_con["policy_years"]= opioid_buyers_con["year"]-2010

con_opi_before=opioid_buyers_con[opioid_buyers_con["policy_years"]<0]
con_opi_after=opioid_buyers_con[opioid_buyers_con["policy_years"]>=0]


opioid_deaths_con=opioid_deaths[opioid_deaths["STNAME"].isin(states2)]
opioid_deaths_con["policy_years"]= opioid_deaths_con["Year"]-2010


con_death_before=opioid_deaths_con[opioid_deaths_con["policy_years"]<0]
con_death_after=opioid_deaths_con[opioid_deaths_con["policy_years"]>=0]

/var/folders/5b/sjkc4b8n0hs41zd03hxr66100000gn/T/ipykernel_47842/4233447948.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  opioid_buyers_con["policy_years"]= opioid_buyers_con["year"]-2010
/var/folders/5b/sjkc4b8n0hs41zd03hxr66100000gn/T/ipykernel_47842/4233447948.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  opioid_deaths_con["policy_years"]= opioid_deaths_con["Year"]-2010


In [82]:
# control buyers pre/post plot

before=alt.Chart(con_opi_before,title="Difference in Difference: Florida vs. Control, Opioids per cap").encode(x="policy_years", y="MME_per_cap")
base=before.transform_regression("policy_years","MME_per_cap").mark_line(color="red") 

fit, before_line = get_reg_fit(
    con_opi_before, yvar="MME_per_cap", xvar="policy_years", alpha=0.05, col="red")

fit, after_line = get_reg_fit(
    con_opi_after, yvar="MME_per_cap", xvar="policy_years", alpha=0.05, col="red")

con_chart_opioid=base+before_line+after_line+sep_line
#con_chart_opioid

In [64]:
diff_fl_opioid=con_chart_opioid+fl_chart_opioid
diff_fl_opioid

alt.LayerChart(...)

In [77]:
# control deaths pre/post plot
fit, before_line = get_reg_fit(
    con_death_before, yvar="deaths_per_cap", xvar="policy_years", alpha=0.05,col="red"
)
fit, after_line = get_reg_fit(
    con_death_after, yvar="deaths_per_cap", xvar="policy_years", alpha=0.05,col="red"
)

before=alt.Chart(con_death_before,title="Difference in Difference: Florida vs. Control, Overdose Deaths per cap").encode(x="policy_years", y="deaths_per_cap")
base=before.transform_regression("policy_years","deaths_per_cap").mark_line(color="#FFAA00") 

con_chart_death=base+before_line+after_line+sep_line

In [80]:
#Difference in difference plot, deaths, FL vs control
diff_fl_death=con_chart_death+fl_chart_death+sep_line
diff_fl_death

alt.LayerChart(...)

# Texas Plots

In [93]:
# creating texas dataframes

opioid_deaths_tx=opioid_deaths[opioid_deaths["STNAME"]=="Texas"]

opioid_deaths_tx["policy_years"]= opioid_deaths_tx["Year"]-2007


tx_death_before=opioid_deaths_tx[opioid_deaths_tx["policy_years"]<0]
tx_death_after=opioid_deaths_tx[opioid_deaths_tx["policy_years"]>=0]



/var/folders/5b/sjkc4b8n0hs41zd03hxr66100000gn/T/ipykernel_47842/1768511342.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  opioid_deaths_tx["policy_years"]= opioid_deaths_tx["Year"]-2007


In [95]:
# Creating texas control dataframes
states = ["Oklahoma","Lousiana","New Mexico"]

opioid_deaths_con=opioid_deaths[opioid_deaths["STNAME"].isin(states)]
opioid_deaths_con["policy_years"]= opioid_deaths_con["Year"]-2007


con_death_before=opioid_deaths_con[opioid_deaths_con["policy_years"]<0]
con_death_after=opioid_deaths_con[opioid_deaths_con["policy_years"]>=0]

/var/folders/5b/sjkc4b8n0hs41zd03hxr66100000gn/T/ipykernel_47842/2473775581.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  opioid_deaths_con["policy_years"]= opioid_deaths_con["Year"]-2007


In [94]:
# texas deaths pre/post plot
before=alt.Chart(tx_death_before,title="Pre-Post Policy: Texas, Overdose Deaths per cap").encode(x="policy_years", y="deaths_per_cap")
base=before.transform_regression("policy_years","deaths_per_cap").mark_line() 

fit, before_line = get_reg_fit(
    tx_death_before, yvar="deaths_per_cap", xvar="policy_years", alpha=0.05
)
fit, after_line = get_reg_fit(
    tx_death_after, yvar="deaths_per_cap", xvar="policy_years", alpha=0.05
)

tx_chart_death=base+before_line+after_line+sep_line
tx_chart_death

alt.LayerChart(...)

In [96]:
# control deaths pre/post plot
fit, before_line = get_reg_fit(
    con_death_before, yvar="deaths_per_cap", xvar="policy_years", alpha=0.05,col="red"
)
fit, after_line = get_reg_fit(
    con_death_after, yvar="deaths_per_cap", xvar="policy_years", alpha=0.05,col="red"
)

before=alt.Chart(con_death_before,title="Difference in Difference: Texas vs. Control, Overdose Deaths per cap").encode(x="policy_years", y="deaths_per_cap")
base=before.transform_regression("policy_years","deaths_per_cap").mark_line(color="#FFAA00") 

con_chart_death=base+before_line+after_line+sep_line

diff_tx_death=con_chart_death+tx_chart_death+sep_line
diff_tx_death

alt.LayerChart(...)

# Washington Plots

In [98]:
# creating Washington dataframes

opioid_deaths_wa=opioid_deaths[opioid_deaths["STNAME"]=="Washington"]

opioid_deaths_wa["policy_years"]= opioid_deaths_wa["Year"]-2012


wa_death_before=opioid_deaths_wa[opioid_deaths_wa["policy_years"]<0]
wa_death_after=opioid_deaths_wa[opioid_deaths_wa["policy_years"]>=0]



/var/folders/5b/sjkc4b8n0hs41zd03hxr66100000gn/T/ipykernel_47842/1298743784.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  opioid_deaths_wa["policy_years"]= opioid_deaths_wa["Year"]-2012


In [101]:
# Creating wa control dataframes
states = ["Idaho","Oregon","Montana"]

opioid_deaths_con=opioid_deaths[opioid_deaths["STNAME"].isin(states)]
opioid_deaths_con["policy_years"]= opioid_deaths_con["Year"]-2012


con_death_before=opioid_deaths_con[opioid_deaths_con["policy_years"]<0]
con_death_after=opioid_deaths_con[opioid_deaths_con["policy_years"]>=0]

/var/folders/5b/sjkc4b8n0hs41zd03hxr66100000gn/T/ipykernel_47842/620745993.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  opioid_deaths_con["policy_years"]= opioid_deaths_con["Year"]-2012


In [102]:
# washington deaths pre/post plot
before=alt.Chart(wa_death_before,title="Pre-Post Policy: Washington, Overdose Deaths per cap").encode(x="policy_years", y="deaths_per_cap")
base=before.transform_regression("policy_years","deaths_per_cap").mark_line() 

fit, before_line = get_reg_fit(
    wa_death_before, yvar="deaths_per_cap", xvar="policy_years", alpha=0.05
)
fit, after_line = get_reg_fit(
    wa_death_after, yvar="deaths_per_cap", xvar="policy_years", alpha=0.05
)

wa_chart_death=base+before_line+after_line+sep_line
wa_chart_death

alt.LayerChart(...)

In [103]:
# control deaths pre/post plot
fit, before_line = get_reg_fit(
    con_death_before, yvar="deaths_per_cap", xvar="policy_years", alpha=0.05,col="red"
)
fit, after_line = get_reg_fit(
    con_death_after, yvar="deaths_per_cap", xvar="policy_years", alpha=0.05,col="red"
)

before=alt.Chart(con_death_before,title="Difference in Difference: Washington vs. Control, Overdose Deaths per cap").encode(x="policy_years", y="deaths_per_cap")
base=before.transform_regression("policy_years","deaths_per_cap").mark_line(color="#FFAA00") 

con_chart_death=base+before_line+after_line+sep_line

diff_wa_death=con_chart_death+wa_chart_death+sep_line
diff_wa_death

alt.LayerChart(...)